In [2]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
import numpy as np
import yfinance as yf
import plotly.express as px
from tinyshift.series import relative_strength_index, entropy_volatility, bollinger_bands, adi_cv, foreca, maximum_achievable_accuracy, sample_entropy, hampel_filter, hurst_exponent
from numpy.random import standard_normal

In [3]:
ticker = 'PETR4.SA'
period = '2y'
data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)
t = np.linspace(0, 10, 1000)
pure_sine = np.sin(2 * np.pi * 1 * t)  # Perfectly forecastable

[*********************100%***********************]  1 of 1 completed


In [4]:
x = np.asarray(data["Close"]["PETR4.SA"], dtype=np.float64)
returns  = np.log(x / np.roll(x, 1))
mean = np.mean(returns)
std = np.std(returns)
norm = (returns - mean) / std #normalized returns

# RSI

In [5]:
vol_rsi = relative_strength_index(data["Close"]["PETR4.SA"], rolling_window=14)

In [6]:
px.line(vol_rsi, title=f"{ticker} Closing Prices").show(fig_type="png")

# Entropy Volatility

In [7]:
vol_ent = entropy_volatility(norm, rolling_window=30)

In [8]:
px.line(vol_ent, title=f"{ticker} Closing Prices").show(fig_type="png")

In [9]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=data.index[61:],
    y=vol_ent,
    name="Volatility Entropy",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=data.index[61:],
    y=vol_rsi[61:],
    name="Volatility RSI",
    yaxis="y3"
))


fig.update_layout(
    yaxis3=dict(
        title="Volatility RSI",
        anchor="free",
        overlaying="y",
        side="right",
        position=1.0,
        showgrid=False
    )
)

fig.update_layout(
    title="Preço PETR4.SA e Volatility Entropy",
    xaxis_title="Data",
    yaxis=dict(title="Preço PETR4.SA"),
    yaxis2=dict(title="Volatility Entropy", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)

fig.show(fig_type="png")

# Bollinger's Band

In [10]:
bb = bollinger_bands(data["Close"]["PETR4.SA"])

In [11]:
data["Close"]["PETR4.SA"][bb]

Date
2023-11-08    24.855495
2023-12-06    25.323149
2023-12-07    25.262672
2024-06-14    28.935343
2024-12-11    34.830196
2025-07-18    30.302549
2025-07-21    30.361217
Name: PETR4.SA, dtype: float64

# Hampel Filter

In [12]:
hf = hampel_filter(data["Close"]["PETR4.SA"], rolling_window=7)

In [13]:
data["Close"]["PETR4.SA"][hf]

Date
2024-05-22    30.161892
2024-07-04    31.138037
2024-10-09    31.938404
2025-02-07    33.884270
Name: PETR4.SA, dtype: float64

# ADI-CV Framework

In [14]:
adi_cv(data["Volume"]["PETR4.SA"])

(np.float64(1.0), np.float64(0.30696726060808555))

# Sampen

In [15]:
sample_entropy(norm, m=3)

np.float64(1.6222001640479518)

In [16]:
sample_entropy(pure_sine, m=2)

np.float64(0.16791509477225003)

# Foreca

In [17]:
foreca(data["Close"]["PETR4.SA"])

0.5862713475212575

In [18]:
foreca(pure_sine)

0.9993976808641186

# Maximum Achievable Accuracy

In [19]:
maximum_achievable_accuracy(norm, m=1)

np.float64(0.797024153007754)

In [20]:
maximum_achievable_accuracy(pure_sine, m=1)

np.float64(0.9855199246783082)

# Hurst's Exponent

In [27]:
hurst_exponent(np.log(data["Close"]["PETR4.SA"]))

(0.6101434657728204, 0.0956610788551536)

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))


fig.show(fig_type="png")

In [186]:
np.random.seed(42)
n_points = 1000
trend = np.linspace(0, 10, n_points)
noise_trend = np.cumsum(standard_normal(n_points) * 0.1)
mean_reversion = np.zeros(n_points)
for t in range(1, n_points):
    mean_reversion[t] = mean_reversion[t-1] * 0.6 + standard_normal() * 0.5
brownian = np.cumsum(standard_normal(n_points))

In [187]:
hurst_exponent(noise_trend)

(0.6419241912818302, 0.000952023886777138)

In [188]:
hurst_exponent(mean_reversion)

(0.3685698852341455, 0.05402692976986088)

In [30]:
hurst_exponent(pure_sine)

(0.8575205638768405, 0.0014654668633357023)